In [116]:
import numpy as np
import pandas as pd

***Dataframe***

In [117]:
samples = 10000
num_features = 2
features = np.random.rand(samples, num_features)
classes = np.random.choice(['d', 'c'], size=samples)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
classes_numeric = label_encoder.fit_transform(classes)
df = pd.DataFrame({'Feature 1': features[:, 0], 'Feature 2': features[:, 1], 'Class': classes_numeric})


***Split the data into training and testing sets***


In [118]:
from sklearn.model_selection import train_test_split
X = df[['Feature 1', 'Feature 2']]
y = df['Class'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [119]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 2)
(2000, 2)
(8000,)
(2000,)


***Desion Tree***

In [120]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
pred = dt.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print(accuracy)


0.514


***Bagging ensemble***

In [121]:
num_of_decisiontree = 10
decision_trees = []
for _ in range(num_estimators):
    index = np.random.choice(len(X_train), len(X_train), replace=True)
    X_sample = X_train.iloc[index]
    y_sample = y_train.iloc[index]
    
    decision_tree = DecisionTreeClassifier(random_state=42)
    decision_tree.fit(X_sampled, y_sampled)
    decision_trees.append(decision_tree)

In [122]:
predictions = []
for tree in decision_trees:
    prediction = tree.predict(X_test)
    predictions.append(prediction)

final_pred = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=predictions)
accuracy = accuracy_score(y_test, final_pred)
print("Bagging Accuracy:", accuracy)

Bagging Accuracy: 0.482
